# Sample Matcher
This module trimmed phenotype/genotype files so that they only have what is in a sample_lookup list, as shown [in this ticket](https://github.com/cumc/xqtl-pipeline/issues/137)

## Input
1. sample_lookup file

    A tab delimated table wit two columns `sample_name_in_pheno(and cov), sample_name_in_geno` that takes only the OVERLAP between these data-set. This will also serve as a sample name matching file if sample names dont agree.
    
2. phenotype_file
    
    A gct like table, Output of bulk_expression_QC or external input before gene_annotation.
    
    Noted, for preping input to normalization, phenotype_trimming need to be run on geneCount and geneTPM each independently
    
3. genotype_file
    
    A plink file, as output of VCF_QC module

## Output
    
1. A gct like phenoFile that is ready to be fed into Normalization/gene_annotation
2. A plink genoFile that is ready to be fed into downstream analysis

## MWE
**FIXME:WIP**

In [ ]:
[global]
# The output directory for generated files. MUST BE FULL PATH
parameter: cwd = path("output")
# Input 1
parameter: sample_participant_lookup = path
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 8
# Software container option
parameter: container = ""
parameter: name = ""

In [ ]:
[phenotype_trimming]
# The molecular phenotype matrix
parameter: phenoFile = path
input: phenoFile,sample_participant_lookup
output: f'{_input[0]:nn}.sample_matched.gct.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = "$[ ]", stderr = f'{_output:nn}.stderr', stdout = f'{_output:nn}.stdout', container = container
    library("dplyr")
    library("readr")
    phenoFile = read_delim($[_input[0]:ar], "\t", col_names = T, comment = "#")
    sample_lookup = read_delim($[_input[1]:ar], "\t" ,col_names = T, comment = "#")
    ## Make phenoFile consistant with sampleLookup, remove samples by select()
    int = intersect(colnames(phenoFile),unlist(sample_lookup[,1]))
    phenoFile_tmp = phenoFile%>%select(c(colnames(phenoFile)[1],all_of(int)))
    ## Add 2 header lines, https://github.com/getzlab/rnaseqc/blob/286f99dfd4164d33014241dd4f3149da0cddf5bf/src/RNASeQC.cpp#L426
    cat(paste("#1.2\n#", nrow(phenoFile_tmp), ncol(phenoFile_tmp) - 2, "\n"), file=$[_output:nr], append=FALSE)
    phenoFile_tmp%>%write_delim($[_output:nr],delim = "\t",col_names = T, append = T)

In [ ]:
[genotype_trimming]
# The path to bed of plink trio
parameter: genoFile = path
input: genoFile, sample_participant_lookup
output: f'{cwd}/{_input[0]:bn}.sample_filtered.bed'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash:  expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', container = container
    paste ${_input[1]} ${_input[1]} | cut -f 2,4 > ${_input[1]}_geno
    plink \
      --bfile ${_input[0]:n} \
      --keep ${_input[1]}_geno \
      --make-bed \
      --out ${_output:n} \
      --threads ${numThreads} \
      --memory ${int(expand_size(mem) * 0.9)/1e6}
    rm ${_input[1]}_geno

The look up table should contain two columns: `sample_id`, `genotype_id`. If the look up table is not provided (file not found) then we will assume the names have already been matched.

In [ ]:
# This workflow extracts overlapping samples for genotype data with phenotype data, and output the filtered sample genotype list as well as sample phenotype list
[genotype_phenotype_sample_overlap]
# A genotype fam file
parameter: genoFile = path
# A phenotype file, can be bed.gz or tsv
parameter: phenoFile = path
# If this file is provided, a genotype/phenotype sample name match will be performed
# It must contain two column names: genotype_id, sample_id
parameter: sample_participant_lookup = path(".")
input: genoFile, phenoFile
output: f'{cwd:a}/{path(_input[2]):bn}.sample_phenotypes.txt', f'{cwd:a}/{path(_input[2]):bn}.sample_genotypes.txt'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = "${ }", stderr = f'{_output[0]:nn}.stderr', stdout = f'{_output[0]:nn}.stdout', container = container

    # Load required libraries
    library(dplyr)
    library(readr)

    # Read data files; let read_delim auto-determine the delimiter
    genoFam <- read_delim(${input[0]:ar}, col_names = FALSE)
    phenoFile <- read_delim(${input[1]:ar}, col_names = TRUE)
    if (file.exists(${sample_participant_lookup:ar})) {
        sample_lookup <- read_delim(${sample_participant_lookup:ar}, "\t", col_names = TRUE)
    } else {
        sample_lookup <- cbind(genoFam$X2, genoFam$X2)
        colnames(sample_lookup) <- c("genotype_id", "phenotype_id")
    }
    sample_lookup <- sample_lookup %>%
    filter(
        genotype_id %in% genoFam$X2,
        sample_id %in% colnames(phenoFile)
    )

    output_genotype <- cbind(genoFam[1, ], sample_lookup$genotype_id)
    output_genotype %>%
    as_tibble() %>%
    write_delim(${output[1]:r}, "\t")

    sample_lookup %>%
    write_delim(${output[0]:r}, "\t")